In [1]:
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.1/108.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from unsloth import FastLanguageModel

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name='/kaggle/input/lora-hse-synth-8b-llama/outputs/checkpoint-140',
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
FastLanguageModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.2: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla P100-PCIE-16GB. Max memory: 15.888 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 6.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers via:
`pip uninstall transformers -y && pip install --upgrade --no-cache-dir "git+https://github.com/huggingface/transformers.git"`
Unsloth 2024.10.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
      

In [3]:
prompt = """
### Instruction:
Вы — опытный ментор по программированию, помогающий студентам учиться на своих ошибках. Ваша задача — давать студенту полезные советы и направлять его, не предоставляя готового решения задачи. Когда студент показывает код, вы должны:
Никогда не давать готовое решение задачи.
Указать на ошибку или недочет, если они есть.
Объяснить, в чем состоит проблема, и предложить направление для исправления.
При необходимости объяснять концепции, которые могут помочь студенту найти решение самостоятельно.
Объясните, что может быть не так в его решении, используя за основу одну или несколько из следующих фраз:
- Обратите внимание на неверный...
- ...необходимо проверить, что...
- Вы забыли поставить...
- Необходимо использовать...
- Вы некорректно...
- Проверьте написание...
- В данном случае не нужно...
- ...неверный синтаксис...
- Ваш код охватывает не все возможные случаи...
- Попробуйте дополнить...
- Вы использовали неверную...
- Ошибка при обращении к...
- Проверьте, что...
- Ваш код использует неверный...

### Input:
{}

### Response:
{}
"""

In [4]:
def get_hint(input_,):
    inputs = tokenizer(
    [
        prompt.format(
            input_,
            '',
        )
    ], return_tensors = 'pt').to('cuda')
    
    outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)
    return tokenizer.batch_decode(outputs)

In [5]:
import pandas as pd

In [6]:
test_solutions = pd.read_excel('/kaggle/input/hse-code-hack/data_v0/for_teams/test/solutions.xlsx')
test_tasks = pd.read_excel('/kaggle/input/hse-code-hack/data_v0/for_teams/test/tasks.xlsx')
test_tests = pd.read_excel('/kaggle/input/hse-code-hack/data_v0/for_teams/test/tests.xlsx')

In [7]:
def combine_data(solutions_, tasks_, tests_):
    solutions, tasks, tests = solutions_.copy(), tasks_.copy(), tests_.copy()

    grouped_tests = tests.groupby('task_id').apply(
        lambda x: "; ".join([
            f"number: {row['number']}, type: {row['type']}, input: {row['input']}, output: {row['output']}"
            for _, row in x.iterrows()
        ])
    ).reset_index()
    grouped_tests = grouped_tests.rename(columns={0: 'tests'})

    merged_df = pd.merge(solutions, tasks, left_on='task_id', right_on='id', suffixes=('_solution', '_task'))
    final_df = pd.merge(merged_df, grouped_tests, how='left', on='task_id')

    return final_df

In [8]:
test = combine_data(test_solutions, test_tasks, test_tests)

/tmp/ipykernel_24/1862003407.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_tests = tests.groupby('task_id').apply(


In [9]:
test_text = test[['description', 'author_solution', 'student_solution', 'tests']]

In [ ]:
import token
import tokenize
from io import StringIO
import re

def remove_comments(code_str):
    multiline_comment_pattern = r'/\*.*?\*/'
    code_str = re.sub(multiline_comment_pattern, '', code_str, flags=re.DOTALL)

    result = []
    prev_toktype = token.INDENT
    last_lineno = -1
    last_col = 0

    tokgen = tokenize.generate_tokens(StringIO(code_str).readline)
    for toktype, ttext, (slineno, scol), (elineno, ecol), ltext in tokgen:
        if slineno > last_lineno:
            last_col = 0
        if scol > last_col:
            result.append(" " * (scol - last_col))

        if toktype == token.STRING and prev_toktype == token.INDENT:
            pass
        elif toktype == tokenize.COMMENT:
            pass
        else:
            result.append(ttext)

        prev_toktype = toktype
        last_col = ecol
        last_lineno = elineno

    return ''.join(result)

test_text['student_solution'] = test_text['student_solution'].apply(remove_comments)

In [10]:
input_template = '''
Условие задачи: {description}

Эталонное решение: {author_solution}

Решение студента: {student_solution}
'''

output_template = '''
Подсказка: {author_comment}

'''

In [11]:
test_df = pd.DataFrame({
    'input': test_text.apply(lambda row: input_template.format(
        description=row['description'],
        author_solution=row['author_solution'],
        student_solution=row['student_solution']
    ), axis=1)
})

In [12]:
from tqdm import tqdm

In [13]:
hints = []
for i in tqdm(range(len(test_df))):
    hint = get_hint(test_df.iloc[i].input)
    hints.append(hint[0].split('Подсказка:')[-1].split('\n\n<|eot_id|>')[0].strip())

100%|██████████| 325/325 [28:42<00:00,  5.30s/it]


In [14]:
import pickle

with open('hints_synthlora_140.pkl', 'wb') as f:
    pickle.dump(hints, f)